In [ ]:
import requests
import time
import os

# Get the data from .env file or from moodle

username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')


url = 'https://www.infraweb-rws.fi/api/auth/login'
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}
payload = {
    "username": username,
    "password": password,
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())

token = response.json()['token']

req_headers = {"Authorization": f"Bearer {token}"}

200
{'token': 'eyJhbGciOiJIUzUxMiJ9.eyJzdWIiOiJqa2xfc3R1ZGVudF9hcGlAbWFyamV0YXMuZmkiLCJ1c2VySWQiOiIzNGZmZDE3MC1lYWI3LTExZWYtOGM5Ny1lZmZkMjkxNGY1NTEiLCJzY29wZXMiOlsiVEVOQU5UX0FETUlOIl0sInNlc3Npb25JZCI6Ijk0NjY0MzE1LWY1M2EtNGFjYi05YjY2LTcyNTlhZWIwNjQ2ZCIsImV4cCI6MTc0MzQwODE5MiwiaXNzIjoiTWFyamV0YXMgQWNhZGVteSBPeSIsImlhdCI6MTc0MzM5OTE5MiwiZmlyc3ROYW1lIjoiSktMIGRhdGEiLCJlbmFibGVkIjp0cnVlLCJpc1B1YmxpYyI6ZmFsc2UsInRlbmFudElkIjoiODI3MzdlYjAtZTQyOS0xMWU5LWFjOGYtZmRjZGVjMmMyNmEyIiwiY3VzdG9tZXJJZCI6IjEzODE0MDAwLTFkZDItMTFiMi04MDgwLTgwODA4MDgwODA4MCJ9.pOhiLv618o_VU3YL_UTObNASQ5n4GKsx5qUUGPZJW3mD2n-_2VJIe5XPUmd4DDSwo5JOrOVg4s0ABtWEWWCypA', 'refreshToken': 'eyJhbGciOiJIUzUxMiJ9.eyJzdWIiOiJqa2xfc3R1ZGVudF9hcGlAbWFyamV0YXMuZmkiLCJ1c2VySWQiOiIzNGZmZDE3MC1lYWI3LTExZWYtOGM5Ny1lZmZkMjkxNGY1NTEiLCJzY29wZXMiOlsiUkVGUkVTSF9UT0tFTiJdLCJzZXNzaW9uSWQiOiI5NDY2NDMxNS1mNTNhLTRhY2ItOWI2Ni03MjU5YWViMDY0NmQiLCJleHAiOjE3NDQwMDM5OTIsImlzcyI6Ik1hcmpldGFzIEFjYWRlbXkgT3kiLCJpYXQiOjE3NDMzOTkxOTIsImlzUHVibGljIjpmYWxzZSwianRp

In [2]:
# get permissions

url = 'http://www.infraweb-rws.fi/api/permissions/allowedPermissions'
response = requests.get(url, headers=req_headers)

data = response.json()

In [3]:
data['userPermissions']['groupPermissions']

{'2309c200-f01b-11ef-b139-295490bf863f': {'entityType': 'DEVICE',
  'operations': ['READ_CREDENTIALS',
   'READ_TELEMETRY',
   'READ_ATTRIBUTES',
   'READ']},
 '3a98fa50-eb4c-11ee-889d-fbb72df75e3e': {'entityType': 'ENTITY_VIEW',
  'operations': ['READ_CREDENTIALS',
   'READ_TELEMETRY',
   'READ_ATTRIBUTES',
   'READ']},
 '3ca1bee0-f70e-11ee-8e4e-3335ad756794': {'entityType': 'DEVICE',
  'operations': ['READ_CREDENTIALS',
   'READ_TELEMETRY',
   'READ_ATTRIBUTES',
   'READ']},
 '1d527fd0-eab9-11ef-8c97-effd2914f551': {'entityType': 'DASHBOARD',
  'operations': ['READ_CREDENTIALS',
   'READ_TELEMETRY',
   'READ_ATTRIBUTES',
   'READ']}}

In [4]:
device_groups = [id for id in data['userPermissions']['groupPermissions'] if data['userPermissions']['groupPermissions'][id]['entityType'] == 'DEVICE']
device_groups

['2309c200-f01b-11ef-b139-295490bf863f',
 '3ca1bee0-f70e-11ee-8e4e-3335ad756794']

In [7]:
devices = []
for group in device_groups:
    time.sleep(1)
    req_url = f"http://www.infraweb-rws.fi/api/entityGroup/{group}/devices?pageSize=100&page=0"
    response = requests.get(req_url, headers=req_headers)
    data = response.json()
    for entity in data['data']:
        if entity['id']['entityType'] == 'DEVICE':
            devices.append(entity['id']['id'])

In [8]:
len(devices)

9

9 devices in total, first 7 are supposedly static, and last 2 are mobile sensors

In [15]:
# test the device id
dev_id = devices[0]

req_url = f"http://www.infraweb-rws.fi/api/device/{dev_id}"

response = requests.get(req_url, headers=req_headers)

response.json()

{'id': {'entityType': 'DEVICE', 'id': '16ceea10-f562-11e9-8fc0-a3372986e528'},
 'createdTime': 1571813724977,
 'tenantId': {'entityType': 'TENANT',
  'id': '82737eb0-e429-11e9-ac8f-fdcdec2c26a2'},
 'customerId': {'entityType': 'CUSTOMER',
  'id': '13814000-1dd2-11b2-8080-808080808080'},
 'name': 'Tie 9 Jyväskylä - ID9014',
 'type': 'DigitrafficWeatherStation',
 'label': None,
 'deviceProfileId': {'entityType': 'DEVICE_PROFILE',
  'id': 'b0021190-51b8-11eb-9a0e-f36c5aeb7b48'},
 'firmwareId': None,
 'softwareId': None,
 'externalId': None,
 'additionalInfo': None,
 'ownerId': {'entityType': 'TENANT',
  'id': '82737eb0-e429-11e9-ac8f-fdcdec2c26a2'},
 'deviceData': {'configuration': {'type': 'DEFAULT'},
  'transportConfiguration': {'type': 'DEFAULT'}}}

In [16]:
req_url = f"http://www.infraweb-rws.fi/api/plugins/telemetry/DEVICE/{dev_id}/values/timeseries"

# check what kind of data is available

response = requests.get(req_url, headers=req_headers)

response.json()

{'ANTURIVIKA': [{'ts': 1652953517000, 'value': '0'}],
 'ASEMAN_STATUS3': [{'ts': 1678263437000, 'value': '1'}],
 'ASEMAN_STATUS_1': [{'ts': 1685078195000, 'value': '31'}],
 'ASEMAN_STATUS_2': [{'ts': 1685078195000, 'value': '31'}],
 'DSC_STATUS': [{'ts': 1678263437000, 'value': '4'}],
 'DSC_VASTAANOTTIMEN_PUHTAUS': [{'ts': 1678263437000, 'value': '0'}],
 'ILMANPAINE': [{'ts': 1685078195000, 'value': '1001.6'}],
 'KASTEPISTE_ERO_ILMA': [{'ts': 1685078195000, 'value': '8.2'}],
 'LUMEN_SYVYYS': [{'ts': 1685078195000, 'value': '0'}],
 'PWD_NAK_TILA': [{'ts': 1685078195000, 'value': '201'}],
 'PWD_STATUS': [{'ts': 1685078195000, 'value': '201'}],
 'PWD_TILA': [{'ts': 1685078195000, 'value': '4'}],
 'SADE_TILA': [{'ts': 1685078195000, 'value': '19'}],
 'VALLITSEVA_SAA': [{'ts': 1685078195000, 'value': '0'}],
 'airtemperature1': [{'ts': 1743399422000, 'value': '3.5'}],
 'airtemperature1change': [{'ts': 1743399422000, 'value': '0.6'}],
 'averagewindspeed': [{'ts': 1685078195000, 'value': '3.4'

In [ ]:
import time

# get the telemetry data for certain period

keys = "friction3"

start_ts = 1711852700 # one year ago
end_ts = int(time.time() * 1000)  # current time in milliseconds
limit = 100000 # how many data points to get

req_url = f"http://www.infraweb-rws.fi/api/plugins/telemetry/DEVICE/{dev_id}/values/timeseries?keys={keys}&startTs={start_ts}&endTs={end_ts}&limit={limit}"

response = requests.get(req_url, headers=req_headers)

response.json()

{'friction3': [{'ts': 1743399422000, 'value': '0.82'},
  {'ts': 1743398822000, 'value': '0.82'},
  {'ts': 1743398222000, 'value': '0.82'},
  {'ts': 1743397622000, 'value': '0.82'},
  {'ts': 1743397023000, 'value': '0.82'},
  {'ts': 1743396422000, 'value': '0.82'},
  {'ts': 1743395822000, 'value': '0.82'},
  {'ts': 1743395222000, 'value': '0.82'},
  {'ts': 1743394622000, 'value': '0.82'},
  {'ts': 1743394022000, 'value': '0.82'},
  {'ts': 1743393422000, 'value': '0.82'},
  {'ts': 1743392822000, 'value': '0.82'},
  {'ts': 1743392222000, 'value': '0.82'},
  {'ts': 1743391623000, 'value': '0.82'},
  {'ts': 1743391022000, 'value': '0.82'},
  {'ts': 1743390423000, 'value': '0.82'},
  {'ts': 1743389822000, 'value': '0.82'},
  {'ts': 1743389222000, 'value': '0.82'},
  {'ts': 1743388622000, 'value': '0.82'},
  {'ts': 1743388022000, 'value': '0.82'},
  {'ts': 1743387423000, 'value': '0.82'},
  {'ts': 1743386822000, 'value': '0.82'},
  {'ts': 1743386222000, 'value': '0.82'},
  {'ts': 174338562200